In [ ]:
from Bio import SeqIO
from Bio.Seq import Seq
from __future__ import division, print_function, absolute_import
import os
import numpy as np
import copy
from sys import path as syspath
import matplotlib.pyplot as plt
import pandas as pd
import pickle
from sklearn import linear_model

In [ ]:
bases = "acgt"
lett_to_index = dict(zip(bases, range(4)))

In [ ]:
is_plus_one = True # True - a brick points to +1; False - points to -40
if is_plus_one: # shift for sequence to adjust +1 or -40
    shift = 40
else:
    shift = 0
    
is_reverse_complement = False # If True, reverse complement fasta seqs, then treat as usual

# For visualisation

max_value = -2.5
min_value = -6

is_high_to_default = False # If True, visualize brickplot values > treshold as default value
treshold = -2.5
default_value = max_value 
color_map = 'hot'

is_prefix_suffix = True # True - add Gs to the begining and end of each sequence to make it equal length and space for matrix


In [ ]:
# Import promoters seqs data
fasta_reader = open(input_fasta_filepath, 'r')

# Initial sequences: fasta -> dict
dict_seqs = {}
max_seq_len = 0
for line in fasta_reader:
    if line.startswith('>'):
        seq_id = line.strip('>').strip()
        print(seq_id)
    else:
        seq = line.strip('"').strip()
        if seq == '':
            continue
        else:
            if is_reverse_complement:
                seq = str(Seq(seq).reverse_complement())
        print('sequence length = '+str(len(seq)))
        if max_seq_len < len(seq):
            max_seq_len = len(seq)
        print(seq)
        dict_seqs[seq_id] = seq
        
fasta_reader.close()

# Sequences unified by length (TO MODIFY?)
# Add artificial 'g's in the begining and the end of all sequences to mak ethem equal by length
unified_seqs_dict = {}
for seq_id in dict_seqs:
    seq = dict_seqs[seq_id]
    if is_prefix_suffix:
        prefix = str('g'*(max_seq_len - len(seq)+shift+5))
        suffix = str('g'*(32+2+shift))
        unified_seqs_dict[seq_id] = (prefix+seq+suffix).lower()
    else:
        unified_seqs_dict[seq_id] = seq.lower()
        
    #print(len(unified_seqs_dict[seq_id]))
        
# Seqs as numbers: 1) list of Int arrays made from Sequences unified by length, 2) list of seq_ids     
num_unified_seqs = []
seq_ids = []
for seq_id in unified_seqs_dict:
    seq_ids.append(seq_id)
    
num_unified_seqs = np.array([np.array([lett_to_index[l] for l in s]) for s in unified_seqs_dict.values()])    
